In [1]:
pip install transformers datasets peft bitsandbytes trl accelerate

In [2]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

In [3]:
# --- 1. CONECTAREA LA GOOGLE DRIVE ---

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# --- 2. Definire Paths ---
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
dataset_path = "/content/drive/MyDrive/DentalData/boli_antrenament_final.json"
output_save_path = "/content/drive/MyDrive/ModeleStomatologie/dental-model-finetuned"

In [5]:
# --- 3. Incarcarea Setului de Date ---
dataset = load_dataset("json", data_files=dataset_path, split="train")

In [6]:
# --- 4. Configurare QLoRA ---
# Quantizam modelul in 4-biti
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
    )

# Definim straturile LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

In [ ]:
# --- 5. Incarcarea Modelului si Tokenizer-ului ---
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16, # Necesar pentru Llama 3
    device_map="auto",
    trust_remote_code=True
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name)
# Llama 3 nu are un pad_token, deci folosim end-of-speech
tokenizer.pad_token = tokenizer.eos_token

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# --- 6. Definirea Trainer-ului ---
# SFTTrainer va:
# 1. Lua setul de date
# 2. Gasi coloana "messages" (indicata de 'dataset_field')
# 3. Aplica automat Llama 3 Chat Template (inclusiv rolul 'system')
# 4. Masca automat "loss-ul" pentru rolurile 'system' si 'user'

trainer = SFTTrainer(
    model=model,
    peft_config=lora_config,
    train_dataset=dataset,
    dataset_field="messages",
    max_seq_length=2048, # A se ajusta
    tokenizer=tokenizer,
    args=TrainingArguments(
        output_dir="./results-necroza-model",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=2,
        num_train_epochs=3,
        learning_rate=1e-4,
        optim="adamw_8bit",
        lr_scheduler_type="cosine",
        fp16=True,
        logging_steps=1,
        save_strategy="epoch",
    ),
)

In [ ]:
# --- 7. Antrenarea ---
print("Se incepe antrenamentul...")
trainer.train()

In [ ]:
# --- 8. Salvarea modelului FINAL in Google Drive ---
print(f"Se salveaza modelul final in: {output_save_path}")
trainer.save_model(output_save_path)

print("Antrenament finalizat!")